In [1]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import glob
import skimage
from skimage import data
from skimage.feature import register_translation
from skimage.feature.register_translation import _upsampled_dft
from scipy.ndimage import fourier_shift
import re
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc


In [2]:
mpl.rcParams['figure.figsize'] = (6, 4)
mpl.rcParams['figure.dpi'] = 150

First we find the folder that holds the images and create a file list based on the files in that folder.

In [3]:
DATA_FOLDER = os.path.join('C:\\', 'Users', 'gt8mar', 'patientg')
print(DATA_FOLDER)

C:\Users\gt8mar\patientg


In [4]:
file_list = glob.glob(os.path.join(DATA_FOLDER, 'Basler_acA1300-200um__23253950__20200219_174245878_*.tiff'))
print(file_list[2])

C:\Users\gt8mar\patientg\Basler_acA1300-200um__23253950__20200219_174245878_100.tiff


In [26]:
pwd

'C:\\Users\\gt8mar\\Documents\\Marcus\\Leuk'

Next we create a list of numbers (num) that correspond to the file names using the regular expression package. We do this because the files will otherwise sort alphabetically, not numerically. We plan to make a dictionary instead. 

In [5]:
myre = re.compile('Basler_acA1300-200um__23253950__20200219_174245878_(?P<num>[\d]+)')

In [27]:
myre.search(file_list[5])['num']

'102'

Here we make a dictionary that has values of our numpy image arrays with keys of the file number. I will later add the date and run number to the dictionary. 

In [8]:
my_images = {}
for i in range(len(file_list)):
    #print(file_list[i])
    num = myre.search(file_list[i])['num']
    my_images[int(num)] = skimage.external.tifffile.imread(file_list[i])
    

In [10]:
sorted_myimages = sorted(my_images)
#print(sorted_myimages)
# print(max(sorted_myimages))



In [11]:
#print(my_images)

Now we have a dictionary where we can call our images by their file number. Next we will check to see how close each file is to it's neighbor. 

In [12]:
shift, error, diffphase = register_translation(my_images[999], my_images[1000], 100)
print(f"Detected subpixel offset (y, x): {shift}")

print(shift[0], shift[1])

Detected subpixel offset (y, x): [-0.01  0.  ]
-0.01 0.0


# This function shows us the offset between frames:

In [12]:




print(len(sorted_myimages))
print(sorted_myimages)
print(range(0, len(sorted_myimages), 1))
[(sorted_myimages[i], sorted_myimages[i+1]) for i in range(0, len(sorted_myimages)-1)]


#     shift, error, diffphase = register_translation(my_images[first], my_images[first+1], 100)
#     if shift[0]>0.5:
#         print(f"Discontinuity between {my_images[first]} and {my_images[first+1]}")
#         break
#     else:
#         print(first, first+1)
#         print(f"Detected subpixel offset (y, x): {shift}")
#         first += 1

    



first file = 991

11
[1, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000]
range(0, 11)


[(1, 991),
 (991, 992),
 (992, 993),
 (993, 994),
 (994, 995),
 (995, 996),
 (996, 997),
 (997, 998),
 (998, 999),
 (999, 1000)]

In [14]:
print(len(sorted_myimages))
#print(sorted_myimages)
print(range(0, len(sorted_myimages), 1))
[(sorted_myimages[i], sorted_myimages[i+1]) for i in range(0, len(sorted_myimages)-1)]

1000
range(0, 1000)


[(1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 9),
 (9, 10),
 (10, 11),
 (11, 12),
 (12, 13),
 (13, 14),
 (14, 15),
 (15, 16),
 (16, 17),
 (17, 18),
 (18, 19),
 (19, 20),
 (20, 21),
 (21, 22),
 (22, 23),
 (23, 24),
 (24, 25),
 (25, 26),
 (26, 27),
 (27, 28),
 (28, 29),
 (29, 30),
 (30, 31),
 (31, 32),
 (32, 33),
 (33, 34),
 (34, 35),
 (35, 36),
 (36, 37),
 (37, 38),
 (38, 39),
 (39, 40),
 (40, 41),
 (41, 42),
 (42, 43),
 (43, 44),
 (44, 45),
 (45, 46),
 (46, 47),
 (47, 48),
 (48, 49),
 (49, 50),
 (50, 51),
 (51, 52),
 (52, 53),
 (53, 54),
 (54, 55),
 (55, 56),
 (56, 57),
 (57, 58),
 (58, 59),
 (59, 60),
 (60, 61),
 (61, 62),
 (62, 63),
 (63, 64),
 (64, 65),
 (65, 66),
 (66, 67),
 (67, 68),
 (68, 69),
 (69, 70),
 (70, 71),
 (71, 72),
 (72, 73),
 (73, 74),
 (74, 75),
 (75, 76),
 (76, 77),
 (77, 78),
 (78, 79),
 (79, 80),
 (80, 81),
 (81, 82),
 (82, 83),
 (83, 84),
 (84, 85),
 (85, 86),
 (86, 87),
 (87, 88),
 (88, 89),
 (89, 90),
 (90, 91),
 (91, 92),
 (92, 93),
 (93,

# This function corrects the offset:

In [17]:
from itertools import product
key_combos = list(product([sorted_myimages[0]], sorted_myimages[1:]))
#print(key_combos)

#for i, j in key_combos:
    #print(i, j)


In [18]:
# Make a new dictionary for the stabilized video files

my_images_new = {}

# Make a list of tuples that match each subsequent file with the first file

from itertools import product
key_combos = list(product([sorted_myimages[0]], sorted_myimages[1:]))
#print(key_combos)

# Iterate through each file and correct it to the base file

for i, j in key_combos:
    shift, error, diffphase = register_translation(my_images[i], my_images[j], 100)
    if shift[0]>0.1:
        print(f"Discontinuity between {my_images[i]} and {my_images[j]} in y")
        break
    if shift[1]>0.1:
        print(f"Discontinuity between {my_images[i]} and {my_images[j]} in x")
        break
    else:
        print(i,j)
        print(f"Detected subpixel offset (y, x): {shift}")
        
        # Take the fourier transform of the file, shift it, and then transform it back.
        
        file_shift = shift
        file_offset = fourier_shift(np.fft.fftn(my_images[j]), file_shift)
        file_offset = np.fft.ifftn(file_offset)
        
        # Create new entries in the stabilized video library
        my_images_new[i] = my_images[i] 
        my_images_new[j] = file_offset.real.astype(int)
        
        shift, error, diffphase = register_translation(my_images_new[i], my_images_new[j], 100)
        print(f"Detected subpixel offset after adjustment (y, x): {shift}")
        print()
        print('------------------')
        print()
       

[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (1, 33), (1, 34), (1, 35), (1, 36), (1, 37), (1, 38), (1, 39), (1, 40), (1, 41), (1, 42), (1, 43), (1, 44), (1, 45), (1, 46), (1, 47), (1, 48), (1, 49), (1, 50), (1, 51), (1, 52), (1, 53), (1, 54), (1, 55), (1, 56), (1, 57), (1, 58), (1, 59), (1, 60), (1, 61), (1, 62), (1, 63), (1, 64), (1, 65), (1, 66), (1, 67), (1, 68), (1, 69), (1, 70), (1, 71), (1, 72), (1, 73), (1, 74), (1, 75), (1, 76), (1, 77), (1, 78), (1, 79), (1, 80), (1, 81), (1, 82), (1, 83), (1, 84), (1, 85), (1, 86), (1, 87), (1, 88), (1, 89), (1, 90), (1, 91), (1, 92), (1, 93), (1, 94), (1, 95), (1, 96), (1, 97), (1, 98), (1, 99), (1, 100), (1, 101), (1, 102), (1, 103), (1, 104), (1, 105), (1, 106), (1, 107), (1, 108), (1, 109), (1, 110), (1, 111), (1, 1

Detected subpixel offset after adjustment (y, x): [0. 0.]

------------------

1 3
Detected subpixel offset (y, x): [ 0.   -0.02]
Detected subpixel offset after adjustment (y, x): [-0.01  0.04]

------------------

1 4
Detected subpixel offset (y, x): [ 0.   -0.02]
Detected subpixel offset after adjustment (y, x): [0.   0.04]

------------------

1 5
Detected subpixel offset (y, x): [-0.01 -0.02]
Detected subpixel offset after adjustment (y, x): [0.01 0.04]

------------------

1 6
Detected subpixel offset (y, x): [ 0.   -0.02]
Detected subpixel offset after adjustment (y, x): [0.   0.04]

------------------

1 7
Detected subpixel offset (y, x): [-0.01 -0.01]
Detected subpixel offset after adjustment (y, x): [0.03 0.04]

------------------

1 8
Detected subpixel offset (y, x): [-0.01 -0.01]
Detected subpixel offset after adjustment (y, x): [0.03 0.03]

------------------

1 9
Detected subpixel offset (y, x): [0. 0.]
Detected subpixel offset after adjustment (y, x): [0. 0.]

-----------

Detected subpixel offset after adjustment (y, x): [0.05 0.02]

------------------

1 65
Detected subpixel offset (y, x): [ 0.   -0.02]
Detected subpixel offset after adjustment (y, x): [0.   0.04]

------------------

1 66
Detected subpixel offset (y, x): [-0.01 -0.02]
Detected subpixel offset after adjustment (y, x): [0.02 0.03]

------------------

1 67
Detected subpixel offset (y, x): [-0.01 -0.02]
Detected subpixel offset after adjustment (y, x): [0.01 0.04]

------------------

1 68
Detected subpixel offset (y, x): [-0.01 -0.02]
Detected subpixel offset after adjustment (y, x): [0.02 0.04]

------------------

1 69
Detected subpixel offset (y, x): [-0.01 -0.01]
Detected subpixel offset after adjustment (y, x): [0.03 0.03]

------------------

1 70
Detected subpixel offset (y, x): [-0.01 -0.03]
Detected subpixel offset after adjustment (y, x): [0.01 0.03]

------------------

1 71
Detected subpixel offset (y, x): [-0.01 -0.02]
Detected subpixel offset after adjustment (y, x): [0.02

Detected subpixel offset after adjustment (y, x): [0.01 0.03]

------------------

1 126
Detected subpixel offset (y, x): [-0.01 -0.04]
Detected subpixel offset after adjustment (y, x): [0.01 0.02]

------------------

1 127
Detected subpixel offset (y, x): [-0.01 -0.03]
Detected subpixel offset after adjustment (y, x): [0.01 0.03]

------------------

1 128
Detected subpixel offset (y, x): [-0.01 -0.04]
Detected subpixel offset after adjustment (y, x): [0.01 0.02]

------------------

1 129
Detected subpixel offset (y, x): [-0.01 -0.03]
Detected subpixel offset after adjustment (y, x): [0.01 0.02]

------------------

1 130
Detected subpixel offset (y, x): [-0.01 -0.03]
Detected subpixel offset after adjustment (y, x): [0.01 0.03]

------------------

1 131
Detected subpixel offset (y, x): [ 0.   -0.03]
Detected subpixel offset after adjustment (y, x): [0.   0.03]

------------------

1 132
Detected subpixel offset (y, x): [-0.01 -0.04]
Detected subpixel offset after adjustment (y, x)

1 186
Detected subpixel offset (y, x): [-15.02 -13.72]
Detected subpixel offset after adjustment (y, x): [-0.02 -0.02]

------------------

1 187
Detected subpixel offset (y, x): [-0.01 -0.04]
Detected subpixel offset after adjustment (y, x): [0.01 0.02]

------------------

1 188
Detected subpixel offset (y, x): [-14.96 -12.74]
Detected subpixel offset after adjustment (y, x): [0.03 0.03]

------------------

1 189
Detected subpixel offset (y, x): [-0.01 -0.03]
Detected subpixel offset after adjustment (y, x): [0.   0.03]

------------------

1 190
Detected subpixel offset (y, x): [-13.03 -14.87]
Detected subpixel offset after adjustment (y, x): [-0.03  0.01]

------------------

1 191
Detected subpixel offset (y, x): [-0.01 -0.03]
Detected subpixel offset after adjustment (y, x): [0.   0.03]

------------------

1 192
Detected subpixel offset (y, x): [-13.03 -14.72]
Detected subpixel offset after adjustment (y, x): [-0.04  0.06]

------------------

1 193
Detected subpixel offset (y,

...

Detected subpixel offset after adjustment (y, x): [-0.03 -0.03]

------------------

1 718
Detected subpixel offset (y, x): [-36.92 -32.09]
Detected subpixel offset after adjustment (y, x): [ 0.06 -0.01]

------------------

1 719
Detected subpixel offset (y, x): [-38.02 -31.74]
Detected subpixel offset after adjustment (y, x): [-0.02  0.02]

------------------

1 720
Detected subpixel offset (y, x): [-37.01 -32.29]
Detected subpixel offset after adjustment (y, x): [-0.01 -0.01]

------------------

1 721
Detected subpixel offset (y, x): [-38.04 -32.46]
Detected subpixel offset after adjustment (y, x): [-0.06 -0.09]

------------------

1 722
Detected subpixel offset (y, x): [-36.96 -31.98]
Detected subpixel offset after adjustment (y, x): [ 0.03 -0.01]

------------------

1 723
Detected subpixel offset (y, x): [-38.02 -32.39]
Detected subpixel offset after adjustment (y, x): [-0.04 -0.08]

------------------

1 724
Detected subpixel offset (y, x): [-36.96 -31.98]
Detected subpixel of

Detected subpixel offset after adjustment (y, x): [0.04 0.02]

------------------

1 777
Detected subpixel offset (y, x): [-36.02 -34.04]
Detected subpixel offset after adjustment (y, x): [-0.03  0.01]

------------------

1 778
Detected subpixel offset (y, x): [-36.98 -32.76]
Detected subpixel offset after adjustment (y, x): [0.02 0.  ]

------------------

1 779
Detected subpixel offset (y, x): [-37.99 -32.61]
Detected subpixel offset after adjustment (y, x): [ 0.   -0.06]

------------------

1 780
Detected subpixel offset (y, x): [-36.97 -33.79]
Detected subpixel offset after adjustment (y, x): [0.02 0.04]

------------------

1 781
Detected subpixel offset (y, x): [-36.07 -33.87]
Detected subpixel offset after adjustment (y, x): [-0.06  0.08]

------------------

1 782
Detected subpixel offset (y, x): [-34.94 -33.93]
Detected subpixel offset after adjustment (y, x): [0.05 0.  ]

------------------

1 783
Detected subpixel offset (y, x): [-36.07 -34.51]
Detected subpixel offset aft

Detected subpixel offset after adjustment (y, x): [0.02 0.  ]

------------------

1 836
Detected subpixel offset (y, x): [-35.   -39.97]
Detected subpixel offset after adjustment (y, x): [ 3.9  -1.87]

------------------

1 837
Detected subpixel offset (y, x): [-32.03 -40.72]
Detected subpixel offset after adjustment (y, x): [-0.02 -0.05]

------------------

1 838
Detected subpixel offset (y, x): [-36.93 -40.84]
Detected subpixel offset after adjustment (y, x): [ 0.06 -0.01]

------------------

1 839
Detected subpixel offset (y, x): [-34.04 -41.76]
Detected subpixel offset after adjustment (y, x): [-0.03  0.  ]

------------------

1 840
Detected subpixel offset (y, x): [-37.02 -41.57]
Detected subpixel offset after adjustment (y, x): [-0.01  0.04]

------------------

1 841
Detected subpixel offset (y, x): [-35.98 -42.24]
Detected subpixel offset after adjustment (y, x): [ 0.02 -0.02]

------------------

1 842
Detected subpixel offset (y, x): [-34.97 -42.89]
Detected subpixel offs

Detected subpixel offset after adjustment (y, x): [-0.03  0.05]

------------------

1 895
Detected subpixel offset (y, x): [-35.89 -47.69]
Detected subpixel offset after adjustment (y, x): [0.11 0.04]

------------------

1 896
Detected subpixel offset (y, x): [-33.03 -46.97]
Detected subpixel offset after adjustment (y, x): [-0.02 -0.1 ]

------------------

1 897
Detected subpixel offset (y, x): [-33.84 -48.78]
Detected subpixel offset after adjustment (y, x): [0.13 0.02]

------------------

1 898
Detected subpixel offset (y, x): [-36.99 -47.74]
Detected subpixel offset after adjustment (y, x): [0.   0.07]

------------------

1 899
Detected subpixel offset (y, x): [-37.99 -48.69]
Detected subpixel offset after adjustment (y, x): [ 0.02 -0.07]

------------------

1 900
Detected subpixel offset (y, x): [-39.   -48.94]
Detected subpixel offset after adjustment (y, x): [0. 0.]

------------------

1 901
Detected subpixel offset (y, x): [-33.91 -50.97]
Detected subpixel offset after a

Detected subpixel offset after adjustment (y, x): [0.01 0.01]

------------------

1 954
Detected subpixel offset (y, x): [-27.02 -51.03]
Detected subpixel offset after adjustment (y, x): [-0.02  0.  ]

------------------

1 955
Detected subpixel offset (y, x): [-27.96 -52.33]
Detected subpixel offset after adjustment (y, x): [0.03 0.  ]

------------------

1 956
Detected subpixel offset (y, x): [-25.06 -52.36]
Detected subpixel offset after adjustment (y, x): [-0.05  0.  ]

------------------

1 957
Detected subpixel offset (y, x): [-28.01 -52.12]
Detected subpixel offset after adjustment (y, x): [-0.01  0.  ]

------------------

1 958
Detected subpixel offset (y, x): [-28.98 -52.05]
Detected subpixel offset after adjustment (y, x): [2.01 1.06]

------------------

1 959
Detected subpixel offset (y, x): [-30.03 -51.89]
Detected subpixel offset after adjustment (y, x): [-0.03  0.02]

------------------

1 960
Detected subpixel offset (y, x): [-25.04 -52.02]
Detected subpixel offset a

In [20]:
print(my_images_new[845])

[[ 91  96  88 ...  99  97  92]
 [ 92  93  95 ...  96  97  97]
 [ 97  91  96 ...  96  97  93]
 ...
 [ 84  92  95 ...  91  94 100]
 [ 96  98  95 ...  91  93  91]
 [ 93  91  91 ...  96  96  96]]


In [21]:
#plt.imshow(my_images_new[1000], cmap='gray')
my_images_new[999].shape

(1024, 1280)

# Write video

In [25]:
# import ffmpeg
# import ffprobe
# from ffprobe import FFProbe


#skvideo.setFFmpegPath("/usr/local/bin")
import skvideo
skvideo.setFFmpegPath("C:\\Users\\gt8mar\\ffmpeg-20200225-36451f9-win64-static\\bin")
import skvideo.io
import skvideo.utils
import skvideo.datasets





values_array = []

for key in my_images_new:
    values_array.append(my_images_new[key])
    
print(np.array(values_array).shape)

values_array = np.array(values_array)

skvideo.io.vwrite("20200219_174245878_patientg_1.mp4", values_array)

(1000, 1024, 1280)


In [24]:
pwd

'C:\\Users\\gt8mar\\Documents\\Marcus\\Leuk'

In [17]:
outputdata = np.random.random(size=(5, 480, 680, 3)) * 255
outputdata = outputdata.astype(np.uint8)

writer = skvideo.io.FFmpegWriter("outputvideo.mp4")
for i in xrange(5):
        writer.writeFrame(outputdata[i, :, :, :])
writer.close()

AssertionError: Cannot find installation of real FFmpeg (which comes with ffprobe).

In [24]:
width = int(1280)
height = int(1024)
FPS = len(my_images_new)
seconds = 1

In [25]:
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
video = cv2.VideoWriter('testing.avi', fourcc, FPS, (width, height))

In [28]:
sorted_my_images_new = list(sorted(my_images_new))
print(sorted_my_images_new)

for i in sorted_my_images_new:
    frame = my_images_new[i]
    #print(frame)
    video.write(frame)
video.release()

[1, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000]


In [ ]:
file1 = skimage.external.tifffile.imread(file_list[0])
file2 = skimage.external.tifffile.imread(file_list[235])

In [ ]:
shift, error, diffphase = register_translation(file1, file2, 100)
print(f"Detected subpixel offset (y, x): {shift}")

In [ ]:
print(type(file1))
print(type(my_images[1]))

In [ ]:
file4 = skimage.external.tifffile.imread(file_list[3])
file4_shift = (-30.92, -45.76)

file4_offset = fourier_shift(np.fft.fftn(file4), file4_shift)
file4_offset = np.fft.ifftn(file4_offset)

shift, error, diffphase = register_translation(file2, file4_offset)
print(f"Detected subpixel offset (y, x): {shift}")

In [ ]:
type(file4_shift)

In [ ]:



shift, error, diffphase = register_translation(image, offset_image)



print(f"Detected pixel offset (y, x): {shift}")

# subpixel precision
shift, error, diffphase = register_translation(image, offset_image, 100)


# Calculate the upsampled DFT, again to show what the algorithm is doing
# behind the scenes.  Constants correspond to calculated values in routine.
# See source code for details.
cc_image = _upsampled_dft(image_product, 150, 100, (shift*100)+75).conj()
ax3.imshow(cc_image.real)
ax3.set_axis_off()
ax3.set_title("Supersampled XC sub-area")


plt.show()

print(f"Detected subpixel offset (y, x): {shift}")